In [1]:
#!c1.8
import pandas as pd
import os
import numpy as np
import pickle
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from catboost import CatBoostRegressor

In [2]:
#!c1.8
pd.set_option("display.max_columns", None)

In [3]:
#!c1.8

mapping = {
    1: "Туристская 2020 год.xlsx",
    2: "Коптевский бул. 2020 год.xlsx",
    3: "Останкино 0 2020 год.xlsx",
    4: "Глебовская 2020 год.xlsx",
    5: "Спиридоновка ул. 2020 год.xlsx",
    6: "Шаболовка 2020.xlsx",
    7: "Академика Анохина 2020.xlsx",
    8: "Бутлерова 2020.xlsx",
    9: "Пролетарский проспект 2020.xlsx",
    10: "Марьино 2020.xlsx"
}

In [4]:
#!c1.8
data = {}

params = {
    "skiprows": [1],
    "engine": "openpyxl"
}

for station_number, filename in mapping.items():
    path = "data/stations/" + filename
    data[station_number] = pd.read_excel(path, **params)

In [5]:
#!c1.8
profiles_dir = "data/ostankino_profile/"

daily_tables = {}

for file in os.scandir(profiles_dir):
    daily_table = pd.read_table(file.path, skiprows=19, decimal=",")
    date = file.name[4:12]
    daily_tables[date] = daily_table

In [6]:
#!c1.8
ost_profile_data = pd.concat(daily_tables)
ost_profile_data["data time"] = pd.to_datetime(ost_profile_data["data time"])
ost_profile_data.drop("Quality", axis=1, inplace = True)
ost_profile_data.rename({
    "data time": "datetime",
    "0": "t_0m",
    "50": "t_50m",
    "100": "t_100m",
    "150": "t_150m",
    "200": "t_200m",
    "250": "t_250m",
    "300": "t_300m",
    "350": "t_350m",
    "400": "t_400m",
    "450": "t_450m",
    "500": "t_500m",
    "550": "t_550m",
    "600": "t_600m",
    "OutsideTemperature": "outside_temperature"
}, axis=1, inplace =True)
ost_profile_data.reset_index(drop=True, inplace=True)
ost_profile_data = ost_profile_data.resample("20min", on="datetime").mean().reset_index()

In [7]:
#!c1.8
ost_253_meteo = pd.read_excel("data/ostankino_meteo.xls", skiprows=2, names=["datetime", "253_wind_direction", "253_wind_speed"])
ost_253_meteo = ost_253_meteo.resample("20min", on="datetime").mean().reset_index()

In [8]:
#!c1.8
ost_data = pd.merge(ost_profile_data, ost_253_meteo, how="inner", on="datetime")

In [9]:
#!c1.8
ost_data

,datetime,t_0m,t_50m,t_100m,t_150m,t_200m,t_250m,t_300m,t_350m,t_400m,t_450m,t_500m,t_550m,t_600m,outside_temperature,253_wind_direction,253_wind_speed
0,2020-01-01 00:00:00,1.8025,2.6400,2.7475,2.3750,1.7575,0.8525,-0.0075,-0.6125,-0.9350,-1.2150,-1.4300,-1.5700,-1.7100,1.7975,300.0,4.00
1,2020-01-01 00:20:00,1.8650,2.7000,2.8150,2.4275,1.7950,0.8800,-0.0225,-0.6525,-0.9850,-1.2675,-1.4850,-1.6200,-1.7575,1.8500,300.0,9.10
2,2020-01-01 00:40:00,1.8550,2.6775,2.8050,2.4400,1.8350,0.9575,0.1200,-0.4625,-0.7775,-1.0475,-1.2500,-1.3700,-1.4975,1.8450,300.0,6.75
3,2020-01-01 01:00:00,1.7950,2.4500,2.5200,2.1825,1.6400,0.8650,0.1375,-0.3550,-0.6000,-0.8300,-0.9925,-1.0875,-1.1775,1.7900,300.0,9.00
4,2020-01-01 01:20:00,1.7425,2.5200,2.5725,2.1850,1.5425,0.6950,-0.1075,-0.6575,-0.9425,-1.1950,-1.3800,-1.4875,-1.5975,1.7475,310.0,4.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26347,2020-12-31 22:20:00,-1.1550,-1.4325,-2.0100,-2.5850,-2.8000,-2.6375,-2.2600,-1.7150,-1.2500,-0.9375,-0.6275,-0.2750,0.0750,-1.1725,170.0,3.35
26348,2020-12-31 22:40:00,-1.3175,-1.6225,-2.1900,-2.6500,-2.7850,-2.6125,-2.2425,-1.7225,-1.2750,-0.9775,-0.7075,-0.4100,-0.1100,-1.2875,175.0,5.35
26349,2020-12-31 23:00:00,-1.4075,-1.7075,-2.2450,-2.7075,-2.8625,-2.7300,-2.3925,-1.9175,-1.5025,-1.2250,-0.9675,-0.6825,-0.3925,-1.4300,170.0,5.05
26350,2020-12-31 23:20:00,-1.6350,-1.9550,-2.5100,-2.9975,-3.1600,-2.9700,-2.5400,-1.9875,-1.5375,-1.2250,-0.9300,-0.6025,-0.2750,-1.6275,165.0,6.60


In [10]:
#!c1.8
# Drop empty cols, rename them, cast datetime to datetime, join with city-level Ostankino data
for k, v in data.items():
    v = v.loc[:, [name for name in v.columns if "Unnamed" not in name]]
    v.dropna(axis=1, how="all", inplace=True)
    v.rename({
        "Дата и время": "datetime",
        "CO": "co",
        "NO2": "no2",
        "NO": "no",
        "PM10": "pm10",
        "PM2.5": "pm25",
        "-T-": "temperature",
        "| V |": "wind_speed",
        "_V_": "wind_direction",
        "Давление": "pressure",
        "Влажность": "humidity",
        "Осадки": "precipitation"
    }, axis=1, inplace=True)
    v["datetime"] = pd.to_datetime(v["datetime"])
    v = pd.merge(v, ost_data, how="inner", on="datetime")
    data[k] = v

In [11]:
#!c1.8
data[1]

,datetime,co,no2,no,pm10,pm25,temperature,wind_speed,wind_direction,pressure,humidity,precipitation,t_0m,t_50m,t_100m,t_150m,t_200m,t_250m,t_300m,t_350m,t_400m,t_450m,t_500m,t_550m,t_600m,outside_temperature,253_wind_direction,253_wind_speed
0,2020-01-01 00:00:00,0.01,0.0170,0.004,0.008,NaN,2.4,0.4,263.0,729.2,81.0,NaN,1.8025,2.6400,2.7475,2.3750,1.7575,0.8525,-0.0075,-0.6125,-0.9350,-1.2150,-1.4300,-1.5700,-1.7100,1.7975,300.0,4.00
1,2020-01-01 00:20:00,0.01,0.0170,0.003,0.007,NaN,2.5,0.6,0.0,729.3,81.0,NaN,1.8650,2.7000,2.8150,2.4275,1.7950,0.8800,-0.0225,-0.6525,-0.9850,-1.2675,-1.4850,-1.6200,-1.7575,1.8500,300.0,9.10
2,2020-01-01 00:40:00,0.00,0.0170,0.003,0.008,NaN,2.5,0.7,238.0,729.4,81.0,NaN,1.8550,2.6775,2.8050,2.4400,1.8350,0.9575,0.1200,-0.4625,-0.7775,-1.0475,-1.2500,-1.3700,-1.4975,1.8450,300.0,6.75
3,2020-01-01 01:00:00,0.00,0.0170,0.003,0.022,NaN,2.5,0.4,237.0,729.5,81.0,NaN,1.7950,2.4500,2.5200,2.1825,1.6400,0.8650,0.1375,-0.3550,-0.6000,-0.8300,-0.9925,-1.0875,-1.1775,1.7900,300.0,9.00
4,2020-01-01 01:20:00,0.00,0.0170,0.003,0.035,NaN,2.4,0.2,229.0,729.6,81.0,NaN,1.7425,2.5200,2.5725,2.1850,1.5425,0.6950,-0.1075,-0.6575,-0.9425,-1.1950,-1.3800,-1.4875,-1.5975,1.7475,310.0,4.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26347,2020-12-31 22:20:00,0.30,0.0438,0.013,NaN,0.029,-0.1,0.4,0.0,748.3,92.0,0.0,-1.1550,-1.4325,-2.0100,-2.5850,-2.8000,-2.6375,-2.2600,-1.7150,-1.2500,-0.9375,-0.6275,-0.2750,0.0750,-1.1725,170.0,3.35
26348,2020-12-31 22:40:00,0.30,0.0428,0.005,NaN,0.029,0.0,0.2,87.0,748.4,91.3,0.0,-1.3175,-1.6225,-2.1900,-2.6500,-2.7850,-2.6125,-2.2425,-1.7225,-1.2750,-0.9775,-0.7075,-0.4100,-0.1100,-1.2875,175.0,5.35
26349,2020-12-31 23:00:00,0.30,0.0426,0.005,NaN,0.031,-0.1,0.3,118.0,748.4,91.0,0.0,-1.4075,-1.7075,-2.2450,-2.7075,-2.8625,-2.7300,-2.3925,-1.9175,-1.5025,-1.2250,-0.9675,-0.6825,-0.3925,-1.4300,170.0,5.05
26350,2020-12-31 23:20:00,0.30,0.0417,0.003,NaN,0.030,-0.4,0.5,212.0,748.3,91.0,0.0,-1.6350,-1.9550,-2.5100,-2.9975,-3.1600,-2.9700,-2.5400,-1.9875,-1.5375,-1.2250,-0.9300,-0.6025,-0.2750,-1.6275,165.0,6.60


In [12]:
#!c1.8

# Split by pollutant
pollutants = ["co", "no2", "no", "pm10", "pm25"]
for k, v in data.items():
    v_parts = {}
    for p in pollutants:
        if p in v.columns:
            cols_to_keep = ["temperature", "wind_speed", "wind_direction",\
                            "pressure", "humidity", "precipitation"] + [p] + list(ost_data.columns)
            v_part = v.loc[:, cols_to_keep]
            v_part.rename({p: "pollutant_concentration"}, axis=1,inplace=True)
            v_parts[p] = v_part
    data[k] = v_parts

In [13]:
#!c1.8
    
# Add separate date and time variables
for n, dict_ in data.items():
    for pollutant, table in dict_.items():
        table["month"] = table["datetime"].dt.month
        table["day"] = table["datetime"].dt.day
        table["day_of_week"] = table["datetime"].dt.weekday
        table["hour"] = table["datetime"].dt.hour
        table.index = pd.Index(table.datetime)
        table.drop("datetime", axis=1, inplace=True)
    data[n][pollutant] = table

# Generate historical features
# Use rolling to capture some previuos and next measures
hist_features = ["temperature", "wind_speed", "wind_direction",\
                            "pressure", "humidity", "precipitation", "pollutant_concentration"]
for n, dict_ in data.items():
    for pollutant, table in dict_.items():
        for timeshift in [*range(1, 25)] + [168]:
            for feature in hist_features:
                if feature not in list(table.columns):
                    continue
                col_name = feature + "_prev_" + str(timeshift) + "h"
                if timeshift == 168:
                    window = 9
                else:
                    window_size = 6
                col_value = table[feature].rolling(window=window_size).mean().shift(3*timeshift)
                table[col_name] = col_value
        data[n][pollutant] = table

# Generate forecast features
forecast_features = ["temperature", "wind_speed", "wind_direction",\
                            "pressure", "humidity", "precipitation"]
for n, dict_ in data.items():
    for pollutant, table in dict_.items():
        for timeshift in range(1, 25):
            for feature in forecast_features:
                col_name = feature + "_forecast_" + str(timeshift) + "h"
                col_value = table[feature].rolling(window=6).mean().shift(-3*timeshift)
                table[col_name] = col_value
        data[n][pollutant] = table

# Generate target [pollution in 1…24 hours]
for n, dict_ in data.items():
    for pollutant, table in dict_.items():
        for timeshift in range(1, 25):
            col_name = "target_" + str(timeshift) + "h"
            col_value = table["pollutant_concentration"].shift(-3*timeshift)
            table[col_name] = col_value
        data[n][pollutant] = table

In [228]:
#!c1.8
metrics = {}
f_imp = {}

for station_number, station_data in data.items():
    metrics_by_pollutant = {}
    f_imp_by_pollutant = {}
    for pollutant_name, pollutant_data in station_data.items():
        target_names = [name for name in pollutant_data.columns if "target" in name]
        train_data = pollutant_data.dropna(axis=0, subset=target_names)
        
        if train_data.shape[0] < 10000:
            print(f"Not enough data for {pollutant_name} on station {station_number}:\
            n of rows in the dataset is {train_data.shape}, skipping.")
        
        X = train_data.drop(target_names, axis=1)
        y = train_data.loc[:, target_names]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=33)
        
        catboost_params = {
            "iterations": 100,
            "verbose": 100,
            "learning_rate": 1,
            "depth": 8,
            "loss_function": "MultiRMSE"}
        model =  CatBoostRegressor(**catboost_params)
        model.fit(X_train, y_train)
        
        # Uncomment if you want to save models        
        # model.save_model(f"pretrained_models/{station_number}_{pollutant_name}.cbm")
        
        predictions = model.predict(X_test)
        
        predictions[predictions < 0] = 0
        
        r2 = r2_score(y_test, predictions, multioutput="raw_values")
        mse = mean_squared_error(y_test, predictions, multioutput="raw_values")
        rmse = model.get_best_score()
        metrics_by_pollutant[pollutant_name] = pd.DataFrame({"r2": r2, "mse": mse, "rmse": rmse["learn"]["MultiRMSE"]})
        
        f_imp_by_pollutant[pollutant_name] = pd.DataFrame({"features": list(X.columns),
                                                           "importance": model.get_feature_importance()})
    metrics[station_number] = pd.concat(metrics_by_pollutant)
    f_imp[station_number] = pd.concat(f_imp_by_pollutant)

0:	learn: 0.9073150	total: 1.73s	remaining: 2m 51s
99:	learn: 0.3512676	total: 2m 37s	remaining: 0us
0:	learn: 0.0525029	total: 1.69s	remaining: 2m 47s
99:	learn: 0.0165735	total: 2m 43s	remaining: 0us
0:	learn: 0.0961284	total: 1.65s	remaining: 2m 43s
99:	learn: 0.0287181	total: 2m 41s	remaining: 0us
0:	learn: 0.0579452	total: 1.38s	remaining: 2m 17s
99:	learn: 0.0130631	total: 2m 11s	remaining: 0us
0:	learn: 0.0285874	total: 1.4s	remaining: 2m 18s
99:	learn: 0.0101401	total: 2m 13s	remaining: 0us
0:	learn: 0.8513051	total: 1.6s	remaining: 2m 38s
99:	learn: 0.4268466	total: 2m 32s	remaining: 0us
0:	learn: 0.0662058	total: 1.61s	remaining: 2m 39s
99:	learn: 0.0235124	total: 2m 42s	remaining: 0us
0:	learn: 0.1117985	total: 1.65s	remaining: 2m 43s
99:	learn: 0.0414155	total: 2m 40s	remaining: 0us
0:	learn: 0.0341396	total: 1.47s	remaining: 2m 25s
99:	learn: 0.0105654	total: 2m 16s	remaining: 0us
0:	learn: 0.7187118	total: 1.75s	remaining: 2m 52s
99:	learn: 0.2816044	total: 2m 45s	remaini

In [160]:
#!c1.8

# Look at the metrics by station
metrics_df.groupby(level=0).mean()

,r2,mse,rmse
1,0.830219,0.002577,0.080254
2,0.790975,0.003159,0.124726
3,0.840173,0.001380,0.073638
4,0.784222,0.003566,0.124009
5,0.779829,0.002112,0.089838
6,0.841342,0.001707,0.083947
7,0.783479,0.002073,0.093104
8,0.795611,0.001121,0.064056
9,0.762355,0.003885,0.116703
10,0.825472,0.001999,0.096859


In [161]:
#!c1.8

# Look at the metrics by pollutant
metrics_df.groupby(level=1).mean()

,r2,mse,rmse
co,0.775808,0.010301,0.339143
no,0.821647,0.000135,0.034863
no2,0.827734,0.000051,0.022781
pm10,0.834285,0.000053,0.020962
pm25,0.745247,0.000014,0.011066


In [162]:
#!c1.8

# Look at the metrics by forecast time [1…24]
metrics_df.groupby(level=2).mean()

,r2,mse,rmse
0,0.827030,0.001952,0.093623
1,0.814935,0.002336,0.093623
2,0.809025,0.002154,0.093623
3,0.804960,0.002260,0.093623
4,0.804655,0.002318,0.093623
5,0.803864,0.002429,0.093623
6,0.802432,0.002301,0.093623
7,0.802438,0.002339,0.093623
8,0.805907,0.002323,0.093623
9,0.804464,0.002318,0.093623


In [226]:
# The most important features
pd.concat(f_imp).reset_index().drop("level_2", axis=1).drop_duplicates(subset="features").sort_values("importance", ascending=False).head(40)

,level_0,level_1,features,importance
403,1,co,wind_speed_forecast_17h,6.534726
389,1,co,precipitation_forecast_14h,5.524454
349,1,co,wind_speed_forecast_8h,5.439922
0,1,co,pollutant_concentration,4.076198
1,1,co,no2,3.925982
37,1,co,pollutant_concentration_prev_1h,3.232708
191,1,co,pollutant_concentration_prev_15h,2.839520
166,1,co,pressure_prev_13h,2.803040
428,1,co,wind_direction_forecast_21h,2.685922
322,1,co,humidity_forecast_3h,2.423114
